In [5]:
#@title Download the dataset
!wget https://cdn.talentsprint.com/aiml/Experiment_related_data/IMDB_Dataset.csv
print("Dataset downloaded successfully!")
!ls | grep ".csv"

Dataset downloaded successfully!


'wget' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup          
from numpy import array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, GlobalMaxPooling1D, Dense, Embedding, LSTM, GRU
import pandas.testing as tm

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
import yaml
import json

from google.colab import drive

# Mount Google Drive with force remount
drive.mount('/soham_apis', force_remount=True)


# Load relevant API Keys
file_path = 'API_KEYS.yml'

with open(file_path, 'r') as file:
    api_keys = yaml.safe_load(file)


# Extract Kaggle username and key
aws_access_key_id = api_keys['AWS']['AWS_ACCESS_KEY_ID']
aws_secret_access_key = api_keys['AWS']['AWS_SECRET_ACCESS_KEY']

ModuleNotFoundError: No module named 'yaml'

In [ ]:
!pip install awscli
!aws s3 cp s3://datasciencedataset/IMDB_Dataset.csv .


movie_reviews = pd.read_csv("/content/IMDB_Dataset.csv")

# Check the shape of the data
movie_reviews.shape

movie_reviews.head()

# Check for null Values
movie_reviews.isnull().sum()

# Let us view one of the reviews
movie_reviews["review"][5]

# View the distribution of positive and negative sentiments in the dataset
sns.countplot(x='sentiment', data = movie_reviews)
plt.show()

# removing the html strips
def strip_html(text):
    # BeautifulSoup is a useful library for extracting data from HTML and XML documents
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

movie_reviews['review'] = movie_reviews['review'].apply(strip_html)

# removing punctuations
def remove_punctuations(text):

    pattern = r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern,'',text)

    # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

    # Removing multiple spaces
    text = re.sub(r'\s+', ' ', text)

    return text

# Remove puntuations
movie_reviews['review'] = movie_reviews['review'].apply(remove_punctuations)

# setting english stopwords
stopword_list = nltk.corpus.stopwords.words('english')
print(stopword_list)

# Exclude 'not' and its other forms from the stopwords list

updated_stopword_list = []

for word in stopword_list:
    if word=='not' or word.endswith("n't"):
        pass
    else:
        updated_stopword_list.append(word)

print(updated_stopword_list)

# removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    # splitting strings into tokens (list of words)
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        # filtering out the stop words
        filtered_tokens = [token for token in tokens if token not in updated_stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in updated_stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# Apply function on review column
movie_reviews['review'] = movie_reviews['review'].apply(remove_stopwords)

movie_reviews.head()

# Convert sentiment labels to integers

movie_reviews['sentiment'] = movie_reviews['sentiment'].apply(lambda x: 1 if x=="positive" else 0)

movie_reviews.head()

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(movie_reviews['review'].values, movie_reviews['sentiment'].values,
                                                    test_size=0.20,
                                                    random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

# Tokenizer class from the keras.preprocessing.text module creates a word-to-index dictionary
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_tok = tokenizer.texts_to_sequences(X_train)
X_test_tok = tokenizer.texts_to_sequences(X_test)

# Find the vocabulary size and perform padding on both train and test set
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train_pad = pad_sequences(X_train_tok, padding='post', maxlen=maxlen, truncating='post')
X_test_pad = pad_sequences(X_test_tok, padding='post', maxlen=maxlen, truncating='post')

print ('number of unique words in the corpus:', vocab_size)

   ---------------------------------------- 0.0/4.4 MB ? eta -:--:--
    --------------------------------------- 0.1/4.4 MB 2.0 MB/s eta 0:00:03
   - -------------------------------------- 0.1/4.4 MB 1.8 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/4.4 MB 1.7 MB/s eta 0:00:03
   -- ------------------------------------- 0.3/4.4 MB 1.4 MB/s eta 0:00:03
   --- ------------------------------------ 0.4/4.4 MB 1.8 MB/s eta 0:00:03
   --- ------------------------------------ 0.4/4.4 MB 1.6 MB/s eta 0:00:03
   ---- ----------------------------------- 0.5/4.4 MB 1.6 MB/s eta 0:00:03
   ----- ---------------------------------- 0.6/4.4 MB 1.6 MB/s eta 0:00:03
   ------ --------------------------------- 0.7/4.4 MB 1.7 MB/s eta 0:00:03
   ------ --------------------------------- 0.7/4.4 MB 1.7 MB/s eta 0:00:03
   ------- -------------------------------- 0.9/4.4 MB 1.7 MB/s eta 0:00:03
   -------- ------------------------------- 0.9/4.4 MB 1.7 MB/s eta 0:00:03
   --------- ------

fatal error: Unable to locate credentials


NameError: name 'pd' is not defined

In [ ]:
EMBEDDING_DIM = 32

print('Build model...')

model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = EMBEDDING_DIM, input_length=maxlen))
model.add(LSTM(units=40,  dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

In [ ]:
history = model.fit(X_train_pad, y_train, batch_size=128, epochs=3, verbose=1, validation_split=0.2)

In [ ]:
print('Testing...')
y_test = np.array(y_test)
score, acc = model.evaluate(X_test_pad, y_test, batch_size=128)

print('Test score:', score)
print('Test accuracy:', acc)

print("Accuracy: {0:.2%}".format(acc))

In [ ]:
#Let us test some  samples
test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

test_samples_tokens = tokenizer.texts_to_sequences(test_samples)
test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen=maxlen)

# predict
pred = model.predict(x=test_samples_tokens_pad)
pred